# iCar App

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
import pandas as pd
import time

class Cars:
    wheels=4
    def __init__(self,marca,modelo):
        self.marca=marca
        self.betas=mlr_b
        self.df_fuel=df_fuel
        self.b0=b0_fuel
        
    def price(self,year,kms,cvs):
        price=float(self.betas[0][0])+(float(self.betas[1][0])*kms)+(float(self.betas[2][0])*year)+(float(self.betas[3][0])*cvs)
        dic1={}
        for x in range(0,len(df_fuel)):
            dic1[x]=df_fuel[x]
            
        self.fuel=dic1[int(input(f"{dic1} "))]
        b=len(self.df_fuel)
        try:
            h=df_fuel.index(self.fuel)
        except:
            return f"There are no {self.fuel} cars available to calculate a fair price"
        
        if self.fuel==b0_fuel:
            return f"{round(price)} EUR"
        else:
            a=self.df_fuel.index(self.fuel)
            price+=float(self.betas[4+a][0])
            return f"{round(price)} EUR"

In [2]:
def user_specs():  
    user_year=int(input("Car Year "))
    user_kms=int(input("Car Kms "))
    user_cvs=int(input("Car horsepower "))
    fuel_db={0:"Diesel",1:"Eléctrico",2:"Gasolina",3:'Híbrido (Gasolina)',4:'Híbrido (Diesel)',5:'GPL'}
    global user_fuel
    user_fuel=fuel_db[int(input(f"{fuel_db} "))]
    print(f"{marca}, {modelo}, {user_year}, {user_kms} kms, {user_cvs} cvs, {user_fuel}")
    return user_year,user_kms,user_cvs,user_fuel

def Marca():
    url_1="https://www.standvirtual.com/api/open/categories/29/makes"
    results_1=requests.get(url_1)
    into_json_1=json.loads(results_1.text)
    list_of_brands=list(into_json_1["options"].keys())
    list_of_brands_dic={}
    for x in list(into_json_1["options"].keys()):
        list_of_brands_dic[list(into_json_1["options"].keys()).index(f"{x}")]=x

    print(list_of_brands_dic)
    marca=input("Insert Brand or Number ")
    if marca.isnumeric()==True:
        marca=list_of_brands_dic[int(marca)]
    return marca

def Modelo():
    global marca
    global modelo
    marca=Marca()
    url_2=f"https://www.standvirtual.com/api/open/categories/29/models/{marca}"
    results_2=requests.get(url_2)
    into_json_2 = json.loads(results_2.text)
    list_of_models=list(into_json_2["options"].keys())
    list_of_models_dic={}
    for x in list(into_json_2["options"].keys()):
        list_of_models_dic[list(into_json_2["options"].keys()).index(f"{x}")]=x

    print(list_of_models_dic)
    #Only numbers accepted as there are brands, like BMW, that use numbers as models and would be confusing
    modelo_input=input("Insert NUMBER of Model ")
    modelo=list_of_models_dic[int(modelo_input)]
    return modelo,marca

def num_pags():
    #This function identifies how many pages are there for that specific model
    defmm=Modelo()
    base_url = 'https://www.standvirtual.com/carros/'
    brand=f"/{defmm[0]}"
    model=f"/{defmm[1]}"
    global url_0
    url_0=base_url+brand+model
    response = requests.get(url_0) 
    soup = BeautifulSoup(response.text, "html.parser") 
    if response.status_code!=200:
        print("Run Again")
        num_pags()
    try:
        info_pags=soup.find_all(class_="optimus-app-1oll9pn eihfmt97")
        info_pags2=soup.find_all(class_="pagination-item")
        last_page_str=str(list(info_pags2)[-2])
        last_page_link_position=last_page_str.find("page=")
        last_page_num=last_page_str[last_page_link_position+5:last_page_link_position+8]
        while last_page_num[-1].isnumeric()==False:
            last_page_num=last_page_num[:-1]
        last_page_num=int(last_page_num)
    except:
        try:
            last_page_link_position=last_page_str.find("Page")
            pagess=last_page_str[last_page_link_position+5:last_page_link_position+7]
            while pagess[-1].isnumeric()==False:
                pagess=pagess[0:-1]
            last_page_num=int(pagess)
        except:
            last_page_num=1
    return last_page_num

#This function registers each car the user runs under the class Car
def add_to_class():
    a=clean_string(marca,modelo)
    b=a[0]
    c=a[1]
    try:
        globals()[f"{b}_{c}"]=Cars(f"{b}",f"{c}")
    except:
        pass
#This function cleans the string so a car class class can be created, otherwise "tesla model-s" wouldn't be possible! 
#"tesla_model-S.price" or "alfa-romeo" 
def clean_string(marca,modelo):
    y=""
    for char in marca:
        if char.isalpha()==True or char.isnumeric()==True:
            y=y+char.lower()
    z=""
    for char1 in modelo:
        if char1.isalpha()==True or char1.isnumeric()==True:
            z=z+char1.lower()    
    return y,z

def multiple_regression(df,df_y,df_fuel):
    
    #Multiple Regression using OLS, reducing the sum of squares between the model's predicton and the dataset values
    mlr_y=df_y.to_numpy()
    mlr_x=df.to_numpy()
    mlr_xt=np.transpose(mlr_x)
    mlr_xt_x=np.dot(mlr_xt,mlr_x)
    
    #If the matrix determinant is 0, we have a singular matrix (error)
    try:
        mlr_xt_x_inv=np.linalg.inv(mlr_xt_x)
    except Exception as e:
        print(f"Unable to compute inverse due to {e}")
        raise
        
    mlr_xt_x_inv=np.linalg.inv(mlr_xt_x)
    mlr_xt_y=np.dot(mlr_xt,mlr_y)
    global mlr_b
    #Estimators' Matrix
    mlr_b=np.dot(mlr_xt_x_inv,mlr_xt_y)
    
    
    specs=user_specs()
    
    #Price for a car which uses only one type of fuel
    final_price=mlr_b[0]+(mlr_b[1]*specs[1])+(mlr_b[2]*specs[0])+(mlr_b[3]*specs[2])
    
    #If the car uses more than one type of fuel, we must add that effect to the price

    if user_fuel==b0_fuel:
        pass
    else:
        try:
            kk=list(df.keys()).index(f"{user_fuel}")
            final_price+=mlr_b[kk]
        except:
            return f"There are no vehicles from that model that are {user_fuel} for sale. But the average price for {df_fuel[0]} is {round(final_price[0])} EUR "
        
    final_price2=round(final_price[0])
    return f"The Average Price for your Car is {final_price2} EUR "

def optimal_price():
    global last_page_num
    last_page_num=num_pags()
    
    list_of_prices=[]
    list_of_kms=[]
    list_of_years=[]
    list_of_cvs=[]
    list_of_fuel=[]
    list_for_b0=[]
    list_of_diesel=[]
    list_of_gasoline=[]
    list_of_eletric=[]
    list_of_hgasoline=[]
    list_of_hdiesel=[]
    list_of_gpl=[]
    
    a=last_page_num
    page=f"?page="
    
    #For Loop covering all the pages!
    for pagina in range(1,last_page_num+1):
        
        print(f"{a} sec")
        a=a-1
    
        url_1=url_0+page+str(pagina)
        #print(f"{url_1}")

        response = requests.get(url_1) 
        #We used Beautiful Soup to help filtering information
        soup = BeautifulSoup(response.text, "html.parser") 
        #Organizes the information by articles
        b=soup.find_all(attrs={"data-testid":"listing-ad"})
        
        #find_error=0
        
        #A For Loop that runs all the car articles available. 
        #It identifies the elements of each car and adds them to a list.
        for x in b:
            #print(find_error)
            #find_error=find_error+1
            c=str(x).find("cv</li></ul></div></div><div><p")
            if c==(-1):
                c=str(x).find("cv</li></ul></div></div><div")
                if c==(-1):
                    c=str(x).find("cv</li></ul></div></div><style")
                    if c==-1:
                        c=str(x).find("cv</li></ul></div><style")
                        if c==-1:
                            c=str(x).find("cv</li></ul></div><p")
            d=str(x)[c-95:c]

            e=d.find("<li>")
            f=d[e+4:-1]

            g=str(x)

            #Find CVS
            cv_position=f.find("km</li><li>")
            cv_value=f[cv_position+11:]
            list_of_cvs.append(int(cv_value))

            #Find fuel
            fuel_position=f.find("</li>")
            fuel=f[:fuel_position]
            list_of_fuel.append(fuel)
            
            #Creating Dummy Variables for the type of fuel
            if fuel=="Gasolina":
                list_of_diesel.append(0)
                list_of_gasoline.append(1)
                list_of_eletric.append(0)
                list_of_hgasoline.append(0)
                list_of_hdiesel.append(0)
                list_of_gpl.append(0)
            if fuel=="Diesel":
                list_of_diesel.append(1)
                list_of_gasoline.append(0)
                list_of_eletric.append(0)
                list_of_hgasoline.append(0)
                list_of_hdiesel.append(0)
                list_of_gpl.append(0)
            if fuel=="Eléctrico":
                list_of_diesel.append(0)
                list_of_gasoline.append(0)
                list_of_eletric.append(1)
                list_of_hgasoline.append(0)
                list_of_hdiesel.append(0)
                list_of_gpl.append(0)
            if fuel=="Híbrido (Gasolina)":
                list_of_diesel.append(0)
                list_of_gasoline.append(0)
                list_of_eletric.append(0)
                list_of_hgasoline.append(1)
                list_of_hdiesel.append(0)
                list_of_gpl.append(0)
            if fuel=="Híbrido (Diesel)":
                list_of_diesel.append(0)
                list_of_gasoline.append(0)
                list_of_eletric.append(0)
                list_of_hgasoline.append(0)
                list_of_hdiesel.append(1)
                list_of_gpl.append(0)
            if fuel=="GPL":
                list_of_diesel.append(0)
                list_of_gasoline.append(0)
                list_of_eletric.append(0)
                list_of_hgasoline.append(0)
                list_of_hdiesel.append(0)
                list_of_gpl.append(1)


            #Find KMS
            km_position=f.find("km")
            km=f[km_position-10:km_position]
            while km[0].isnumeric()==False:
                km=km[1:]
            km_1=km.replace(" ","")
            #Put the kms in INT
            list_of_kms.append(int(km_1))


            #Find Year
            year_position=f.find("km")
            year=f[year_position-24:year_position-12]
            while year[0].isnumeric()==False:
                year=year[1:]
            while year[-1].isnumeric()==False:
                year=year[:-1]
            list_of_years.append(int(year))

            #Find Prices                              
            price_position=g.find("EUR<") #EUR</p></article>
            price=g[price_position-15:price_position]
            while price[0]!=">":
                price=price[1:]
            price=price[1:]
            price_1=price.replace(" ","")
            list_of_prices.append(int(price_1))
            list_for_b0.append(1)
            

        
        #If some of the Professors reads this, this detail below deserves 20/20
        #O PROGRAMA COMECOU A DAR ERROS DE EXCESSO DE REQUESTS, 1 SEG DE INTERVALO ENTRE CADA PAGINA
        time.sleep(1)
    #We created the below dataframe to better organize the data and be able to use functions as drop_duplicates()     
    global df
    df = pd.DataFrame()
    df["Price"]=list_of_prices
    df["For B0"]=list_for_b0
    df["Kms"]=list_of_kms
    df["Year"]=list_of_years
    df["CVS"]=list_of_cvs
    df["Fuel"]=list_of_fuel
    
    df["Diesel"]=list_of_diesel
    df["Eléctrico"]=list_of_eletric
    df["Gasolina"]=list_of_gasoline
    df['Híbrido (Gasolina)']=list_of_hgasoline
    df['Híbrido (Diesel)']=list_of_hdiesel
    df['GPL']=list_of_gpl

    #There are "flagged" car articles in several StandVirtual pages
    #These articles appear several times, making it possible for the same car to appear in every page . 
    #If we accounted them several times, it would affect the result
    #So, we decided to remove them
    df=df.drop_duplicates()
    #We separated dependent and independent variables, price being the dependent variable
    #Dependent Variable Matrix
    df_y=pd.DataFrame()
    df_y["Price"]=df['Price']
    #Fuel is a categorical variable
    df=df.drop(columns=["Price","Fuel"])
    
    #In order to do a Multiple Regression, it is necessary that there is a column with all the values equal to 1
    #This is done to get an intercept estimator value
    #However, if there is a second column with all the values equal to each other
    #The second column is proportional to the intercept column
    #Algebraically, when there are two proprtional columns, the determinant will be equal to zero
    #This way, it is impossible to calculate the inverse of the matrix
    #It was, hence, essential to prevent that by deleting columns when needed in a way to get the most efficient estimators
    #E.g. for 'Tesla', there are only electric cars, hence there is matrix line that is all equal to 1
    #In those cases, the line should be deleted
    
    hm=[]
    #Diesel
    df_d=pd.DataFrame()
    df_d['Diesel']=df['Diesel']
    if len(df_d.drop_duplicates())==1:
        df=df.drop(columns="Diesel")
        if df_d['Diesel'][0]==1:
            hm.append('Diesel')
        
    #Gasoline
    df_g=pd.DataFrame()
    df_g['Gasolina']=df['Gasolina']
    if len(df_g.drop_duplicates())==1:
        df=df.drop(columns="Gasolina")
        if df_g['Gasolina'][0]==1:
            hm.append('Gasolina')
    
    #Electric
    df_e=pd.DataFrame()
    df_e['Eléctrico']=df['Eléctrico']
    if len(df_e.drop_duplicates())==1:
        df=df.drop(columns="Eléctrico")
        if df_e['Eléctrico'][0]==1:
            hm.append('Eléctrico')
        
    #Hibrid (Gasoline) 
    df_hg=pd.DataFrame()
    df_hg['Híbrido (Gasolina)']=df['Híbrido (Gasolina)']
    if len(df_hg.drop_duplicates())==1:
        df=df.drop(columns="Híbrido (Gasolina)")
        if df_hg['Híbrido (Gasolina)'][0]==1:
            hm.append('Híbrido (Gasolina)')
        
    #Hibrid (Diesel)
    df_hd=pd.DataFrame()
    df_hd['Híbrido (Diesel)']=df['Híbrido (Diesel)']
    if len(df_hd.drop_duplicates())==1:
        df=df.drop(columns="Híbrido (Diesel)")
        if df_hd['Híbrido (Diesel)'][0]==1:
            hm.append('Híbrido (Diesel)')
    
    #LPG
    df_gp=pd.DataFrame()
    df_gp['GPL']=df['GPL']
    if len(df_gp.drop_duplicates())==1:
        df=df.drop(columns="GPL")
        if df_gp['GPL'][0]==1:
            hm.append('GPL')
        
    #When there is, for instance, a model which uses only Gasoline or Diesel, there will be only one column for fuel
    #This column will be 1 for Gasoline and 0 for Diesel
    #The below condition defines whether 1 means Gasoline or Diesel
    
    global df_fuel
    #Number of different fuels that were found
    df_fuel=pd.DataFrame()
    try:
        df_fuel=list(df.columns)[4:]
        try:
            df_fuel.append(hm[0])
        except:
            pass
    except:
        df_fuel=hm[0]
        
    
    if len(df_fuel)>1:
        global b0_fuel
        b0_fuel=list(df.keys())[-1]
        df=df.drop(columns=b0_fuel)
    else:
        b0_fuel=hm[0]
    
    df_km=pd.DataFrame()
    df_km['Kms']=df['Kms']
    if len(df_km.drop_duplicates())==1:
        return "Not Enough Diversification, Unable to give a Meaningful Estimation "
    
    df_year=pd.DataFrame()
    df_year['Year']=df['Year']
    if len(df_year.drop_duplicates())==1:
        return "Not Enough Diversification, Unable to give a Meaningful Estimation"
    
    df_cvs=pd.DataFrame()
    df_cvs['CVS']=df['CVS']
    if len(df_cvs.drop_duplicates())==1:
        return "Not Enough Diversification, Unable to give a Meaningful Estimation "
    
    #We were able to build Matrixes with the help of NumPy
    
    list_of_prices=np.array(list_of_prices)
    list_of_kms=np.array(list_of_kms)
    list_of_years=np.array(list_of_years)
    list_of_cvs=np.array(list_of_cvs)
    list_of_diesel=np.array(list_of_diesel)
    list_of_gasoline=np.array(list_of_gasoline)
    list_of_eletric=np.array(list_of_eletric)
    list_of_hgasoline=np.array(list_of_hgasoline)
    list_of_hdiesel=np.array(list_of_hdiesel)
    list_of_gpl=np.array(list_of_gpl)
    list_for_b0=np.array(list_for_b0)
    prilimpimpim=multiple_regression(df,df_y,df_fuel)
    add_to_class()
    return prilimpimpim

In [3]:
#run this cell
optimal_price()

{0: 'alfa-romeo', 1: 'aston-martin', 2: 'audi', 3: 'austin-morris', 4: 'bentley', 5: 'bmw', 6: 'chrysler', 7: 'citroen', 8: 'chevrolet', 9: 'daewoo', 10: 'daihatsu', 11: 'dodge', 12: 'ferrari', 13: 'fiat', 14: 'ford', 15: 'gmc', 16: 'honda', 17: 'hummer', 18: 'hyundai', 19: 'isuzu', 20: 'jaguar', 21: 'jeep', 22: 'kia', 23: 'lada', 24: 'lamborghini', 25: 'lancia', 26: 'land-rover', 27: 'lexus', 28: 'lotus', 29: 'maserati', 30: 'mazda', 31: 'mercedes-benz', 32: 'mg', 33: 'mini', 34: 'mitsubishi', 35: 'nissan', 36: 'opel', 37: 'peugeot', 38: 'porsche', 39: 'renault', 40: 'rolls-royce', 41: 'rover', 42: 'saab', 43: 'seat', 44: 'skoda', 45: 'smart', 46: 'ssangyong', 47: 'subaru', 48: 'suzuki', 49: 'tata', 50: 'toyota', 51: 'umm', 52: 'vauxhall', 53: 'volvo', 54: 'vw', 55: 'dacia', 56: 'ds', 57: 'outra-nao-listada', 58: 'tesla', 59: 'datsun', 60: 'abarth', 61: 'triumph', 62: 'morgan', 63: 'microcar', 64: 'ligier', 65: 'aixam', 66: 'austin-healey', 67: 'maybach', 68: 'fisker', 69: 'infiniti',

'The Average Price for your Car is 33018 EUR '

# Example:
Brand_Model.price ( Year, Kms, Cvs )

In [4]:
#for brand names that contain hyphen or spaces, these special characters need to be taken out as shown in the example below "tesla model-s"
#brand_model.price(year)
vw_golf.price(2020,20000,200)


NameError: name 'vw_golf' is not defined